In [2]:
import sys
sys.path.append('../../')
sys.path.append('../')
import time
import pnn

In [3]:
import torch_tensorrt


/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch_tensorrt/fx/tracer/acc_tracer/acc_ops.py:895: UserWarning: Unable to import torchvision related libraries.: No module named 'torchvision'. Please install torchvision lib in order to lower stochastic_depth
  warnings.warn(
Unable to import quantization op. Please install modelopt library (https://github.com/NVIDIA/TensorRT-Model-Optimizer?tab=readme-ov-file#installation) to add support for compiling quantized models
TensorRT-LLM is not installed. Please install TensorRT-LLM or set TRTLLM_PLUGINS_PATH to the directory containing libnvinfer_plugin_tensorrt_llm.so to use converters for torch.distributed ops


[02/11/2025-10:39:15] [TRT] [W] Functionality provided through tensorrt.plugin module is experimental.


In [4]:
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.fx import subgraph_rewriter, symbolic_trace
import utils
from torch.fx import Proxy, Graph, GraphModule
from torch.fx.passes.utils.matcher_utils import SubgraphMatcher

replace success!


In [5]:
from torch.profiler import profile, record_function, ProfilerActivity
import time
import torch._dynamo as dynamo


In [6]:
import torch._dynamo
torch._dynamo.config.suppress_errors = True

In [7]:
def calculate_mean_and_variance_manual(data):
    n = len(data)
    mean = sum(data) / n
    variance = sum((x - mean) ** 2 for x in data) / n
    return mean, variance

In [33]:
class PatternClass(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self,x):
      empty_like_tensor = torch.ones((4096,100,32))
      result_x = x[0,:] - x[0,:]
      empty_like_tensor[:,] = result_x
      reorder_x =  empty_like_tensor[:,l_100,:]
      return torch.sum(reorder_x, dim = 1)

In [34]:
class PatternClass1(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self,x):
      empty_like_tensor = torch.empty((4096,100,32))
      result_x = x[0,:] - x[0,:]
      empty_like_tensor[:,] = result_x
      # reorder_x =  empty_like_tensor[:,(24, 25, 12, 4, 16, 11, 2, 28, 30, 21, 14, 27, 15, 31, 9, 10, 0, 5, 22, 19, 29, 26, 8, 7, 1, 6, 13, 18, 3, 20, 17, 23),:]
      return torch.sum(empty_like_tensor, dim = 1)

In [35]:
class PatternClass2(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self,x):
      x * x
      # reorder_x =  empty_like_tensor[:,(24, 25, 12, 4, 16, 11, 2, 28, 30, 21, 14, 27, 15, 31, 9, 10, 0, 5, 22, 19, 29, 26, 8, 7, 1, 6, 13, 18, 3, 20, 17, 23),:]
      return torch.sum(x * x, dim = 1)

In [36]:
compile = torch.compile(PatternClass())

In [37]:
t = torch.randint(low=0, high=88, size=(4096 ,100,32))

In [40]:
total_time = []
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
      for i in range(100):
          start_time = time.time()  # 开始计时
          with torch.no_grad():
            soutput = compile(t)
          end_time = time.time()  # 结束计时
          
          # 计算并打印函数执行所需的时间
          elapsed_time = end_time - start_time
          total_time.append(elapsed_time * 1000)
      print(calculate_mean_and_variance_manual(total_time))

(0.6078457832336426, 0.04882914310542219)


In [13]:
compile = torch.compile(PatternClass(), backend="openvino")

In [16]:
total_time = []
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
      for i in range(100):
          start_time = time.time()  # 开始计时
          with torch.no_grad():
            soutput = compile(t)
          end_time = time.time()  # 结束计时
          
          # 计算并打印函数执行所需的时间
          elapsed_time = end_time - start_time
          total_time.append(elapsed_time * 1000)
      print(calculate_mean_and_variance_manual(total_time))

(25.180869102478027, 3.9181262520514792)


In [39]:
torch._dynamo.list_backends()

['cudagraphs', 'inductor', 'onnxrt', 'openvino', 'openxla', 'tvm']

In [40]:
compile = torch.compile(PatternClass(), backend="cudagraphs")

In [42]:
total_time = []
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
      for i in range(100):
          start_time = time.time()  # 开始计时
          with torch.no_grad():
            soutput = compile(t)
          end_time = time.time()  # 结束计时
          
          # 计算并打印函数执行所需的时间
          elapsed_time = end_time - start_time
          total_time.append(elapsed_time * 1000)
      print(calculate_mean_and_variance_manual(total_time))

(26.095049381256104, 7.956240326910802)


In [41]:
compile = torch.compile(PatternClass1())

In [47]:
total_time = []
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
      for i in range(100):
          start_time = time.time()  # 开始计时
          with torch.no_grad():
            soutput = compile(t)
          end_time = time.time()  # 结束计时
          
          # 计算并打印函数执行所需的时间
          elapsed_time = end_time - start_time
          total_time.append(elapsed_time * 1000)
      print(calculate_mean_and_variance_manual(total_time))

(0.21811962127685547, 0.004303291257201636)


In [20]:
compile = torch.compile(PatternClass1(), backend="openvino")

In [23]:
total_time = []
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
      for i in range(100):
          start_time = time.time()  # 开始计时
          with torch.no_grad():
            soutput = compile(t)
          end_time = time.time()  # 结束计时
          
          # 计算并打印函数执行所需的时间
          elapsed_time = end_time - start_time
          total_time.append(elapsed_time * 1000)
      print(calculate_mean_and_variance_manual(total_time))

(20.279502868652344, 2.6546979233899037)


In [43]:
compile = torch.compile(PatternClass1(), backend="cudagraphs")

In [45]:
total_time = []
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
      for i in range(100):
          start_time = time.time()  # 开始计时
          with torch.no_grad():
            soutput = compile(t)
          end_time = time.time()  # 结束计时
          
          # 计算并打印函数执行所需的时间
          elapsed_time = end_time - start_time
          total_time.append(elapsed_time * 1000)
      print(calculate_mean_and_variance_manual(total_time))

(19.095818996429443, 5.34275703450362)


In [67]:
compile = torch.compile(PatternClass2())

In [74]:
total_time = []
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
      for i in range(100):
          start_time = time.time()  # 开始计时
          with torch.no_grad():
            soutput = compile(t)
          end_time = time.time()  # 结束计时
          
          # 计算并打印函数执行所需的时间
          elapsed_time = end_time - start_time
          total_time.append(elapsed_time * 1000)
      print(calculate_mean_and_variance_manual(total_time))

(1.130509376525879, 0.2827468068744565)


In [24]:
compile = torch.compile(PatternClass2(), backend="openvino")

In [37]:
total_time = []
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
      for i in range(100):
          start_time = time.time()  # 开始计时
          with torch.no_grad():
            soutput = compile(t)
          end_time = time.time()  # 结束计时
          
          # 计算并打印函数执行所需的时间
          elapsed_time = end_time - start_time
          total_time.append(elapsed_time * 1000)
      print(calculate_mean_and_variance_manual(total_time))

(6.178276538848877, 1.1584116421715862)


In [65]:
compile = torch.compile(PatternClass2(), backend="cudagraphs")

In [66]:
total_time = []
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
      for i in range(100):
          start_time = time.time()  # 开始计时
          with torch.no_grad():
            soutput = compile(t)
          end_time = time.time()  # 结束计时
          
          # 计算并打印函数执行所需的时间
          elapsed_time = end_time - start_time
          total_time.append(elapsed_time * 1000)
      print(calculate_mean_and_variance_manual(total_time))

skipping cudagraphs due to skipping cudagraphs due to cpu device (arg0_1). Found from : 
   File "/tmp/ipykernel_3585688/2837069808.py", line 8, in forward
    return torch.sum(x * x, dim = 1)



KeyboardInterrupt: 

In [62]:
compile = torch.compile(PatternClass2(), backend="tensorrt")

In [64]:
total_time = []
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
      for i in range(100):
          start_time = time.time()  # 开始计时
          with torch.no_grad():
            soutput = compile(t)
          end_time = time.time()  # 结束计时
          
          # 计算并打印函数执行所需的时间
          elapsed_time = end_time - start_time
          total_time.append(elapsed_time * 1000)
      print(calculate_mean_and_variance_manual(total_time))

(30.812289714813232, 2.842436174313434)


In [11]:
torch.onnx.export(PatternClass2(),
                  t,
        '/home/yssun/pytorch-fm/torchfm/model/test_fx/exp/model_repo/test/ori.onnx',
                  input_names=['input'],
                  output_names=['output'],
                  )

In [12]:
torch.onnx.export(PatternClass1(),
                  t,
        '/home/yssun/pytorch-fm/torchfm/model/test_fx/exp/model_repo/test/modify_noreorder.onnx',
                  input_names=['input'],
                  output_names=['output'],
                  )

In [66]:
torch.onnx.export(PatternClass(),
                  t,
        '/home/yssun/pytorch-fm/torchfm/model/test_fx/exp/model_repo/test/modify_reorder.onnx',
                  input_names=['input'],
                  output_names=['output'],
                  )

In [67]:
torch.save(PatternClass(),'/home/yssun/pytorch-fm/torchfm/model/test_fx/exp/model_repo/test/modify_reorder.pt',)

In [91]:
l = [i for i  in range(100) if i != 1 and i != 2]

In [92]:
l

[0,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99]

In [116]:
l = [i for i in range(100)]
l[0] = 1
l[1] = 2
l[2] = 0

In [137]:
class TestClass(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self,x):
      empty_like_tensor = torch.ones((4096,100,32))
      result_x = x[0,(1,2),:] * x[0,(1,2),:]
      empty_like_tensor[:,:2,:] = result_x
      empty_like_tensor[:,2:,:] = x[0,2:,:] * x[0,2:,:]
    #   reorder_x =  empty_like_tensor[:,l_100,:]
      return torch.sum(empty_like_tensor, dim = 1)

In [140]:
compile = torch.compile(TestClass())

In [131]:
t = torch.randint(low=0, high=88, size=(4096 ,100,32))

In [136]:
total_time = []
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
      for i in range(100):
          start_time = time.time()  # 开始计时
          with torch.no_grad():
            soutput = compile(t)
          end_time = time.time()  # 结束计时
          
          # 计算并打印函数执行所需的时间
          elapsed_time = end_time - start_time
          total_time.append(elapsed_time * 1000)
      print(calculate_mean_and_variance_manual(total_time))

(0.3200984001159668, 0.1526529321097314)


In [240]:
class TestClass(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self,x):
      empty_like_tensor = torch.ones((4096,100,32))
      result_x = x[0,(1,2),:] * x[0,(1,2),:]
      empty_like_tensor[:,:2,:] = result_x
      empty_like_tensor[:,2:,:] = x[0,2:,:] * x[0,2:,:]
    #   reorder_x =  empty_like_tensor[:,l_100,:]
      return torch.sum(empty_like_tensor[:,l_100,:], dim = 1)

In [248]:
l_100 = [i for i in range(100)]

In [254]:
@torch.compile()
def f_modify(x):
  empty_like_tensor = torch.ones((4096,100,32))
  result_x = x[0,(1,2),:] * x[0,(1,2),:]
  empty_like_tensor[:,:2,:] = result_x
  empty_like_tensor[:,2:,:] = x[0,2:,:] * x[0,2:,:]
#   reorder_x =  empty_like_tensor[:,l_100,:]
  return torch.sum(empty_like_tensor, dim = 1)

In [250]:
f_modify(torch.empty([4096,100,32]))

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [253]:
total_time = []
t = torch.empty([4096,100,32])
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
      for i in range(100):
          start_time = time.time()  # 开始计时
          with torch.no_grad():
            soutput = f_modify(t)
          end_time = time.time()  # 结束计时
          
          # 计算并打印函数执行所需的时间
          elapsed_time = end_time - start_time
          total_time.append(elapsed_time * 1000)
      print(calculate_mean_and_variance_manual(total_time))

(1.177678108215332, 0.11754041158837936)


In [224]:
l_100 = [i for i in range(100)]

In [230]:
l_100[0], l_100[3] = l_100[3], l_100[0]

In [255]:
@torch.compile()
def f_modify_cuda(x):
  empty_like_tensor = torch.ones((4096,100,32),device='cuda')
  result_x = x[0,(1,2),:] * x[0,(1,2),:]
  empty_like_tensor[:,:2,:] = result_x
  empty_like_tensor[:,2:,:] = x[0,2:,:] * x[0,2:,:]
#   reorder_x =  empty_like_tensor[:,l_100,:]
  return torch.sum(empty_like_tensor[:,l_100,:], dim = 1)

In [258]:
total_time = []
t = torch.empty([4096,100,32],device='cuda')
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
      for i in range(100):
          start_time = time.time()  # 开始计时
          with torch.no_grad():
            soutput = f_modify_cuda(t)
          end_time = time.time()  # 结束计时
          
          # 计算并打印函数执行所需的时间
          elapsed_time = end_time - start_time
          total_time.append(elapsed_time * 1000)
      print(calculate_mean_and_variance_manual(total_time))

(0.14787912368774414, 0.17312163137717107)


In [261]:
total_time = []
t = torch.empty([4096,100,32],device='cuda')
with profile(activities=[torch.profiler.ProfilerActivity.CUDA,ProfilerActivity.CPU], record_shapes=True,
        profile_memory=True,  # 记录内存使用情况
    with_stack=True,  # 记录代码堆栈信息
    with_flops=True,  # 记录FLOPs
    on_trace_ready=torch.profiler.tensorboard_trace_handler('./log_single') ) as prof:
    with record_function("model_inference"):
      for i in range(100):
          start_time = time.time()  # 开始计时
          with torch.no_grad():
            soutput = f_modify_cuda(t)
          end_time = time.time()  # 结束计时
          
          # 计算并打印函数执行所需的时间
          elapsed_time = end_time - start_time
          total_time.append(elapsed_time * 1000)
      print(calculate_mean_and_variance_manual(total_time))

(0.1826000213623047, 0.270089240393645)


In [172]:
@torch.compile
def f(x):
#   reorder_x =  empty_like_tensor[:,l_100,:]
  return torch.sum(x * x, dim = 1)

In [174]:
f(torch.empty([4096,100,32],device = "cuda"))

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')

In [178]:
total_time = []
t = torch.empty([4096,100,32],device="cuda")
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
      for i in range(100):
          start_time = time.time()  # 开始计时
          with torch.no_grad():
            soutput = f(t)
          end_time = time.time()  # 结束计时
          
          # 计算并打印函数执行所需的时间
          elapsed_time = end_time - start_time
          total_time.append(elapsed_time * 1000)
      print(calculate_mean_and_variance_manual(total_time))

(0.12805700302124023, 0.019240927593955348)


In [217]:
total_time = []
t = torch.empty([4096,100,32],device="cpu")
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
      for i in range(100):
          start_time = time.time()  # 开始计时
          with torch.no_grad():
            soutput = f(t)
          end_time = time.time()  # 结束计时
          
          # 计算并打印函数执行所需的时间
          elapsed_time = end_time - start_time
          total_time.append(elapsed_time * 1000)
      print(calculate_mean_and_variance_manual(total_time))

(0.37773847579956055, 0.028915579122212876)


# 正式测试

## 基准

In [101]:
@torch.compile
def f(x):
#   reorder_x =  empty_like_tensor[:,l_100,:]
  return torch.sum(x * x, dim = 1)

### CPU

In [9]:
f(torch.empty([4096,100,32],device = "cpu"))

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [ ]:
f(torch.empty([4096,100,32],device = "cpu"))
l_time = []
t = torch.empty([4096,100,32],device="cpu")
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
      for i in range(100):
          start_time = time.time()  # 开始计时
          with torch.no_grad():
            soutput = f(t)
          end_time = time.time()  # 结束计时
          
          # 计算并打印函数执行所需的时间
          elapsed_time = end_time - start_time
          total_time.append(elapsed_time * 1000)
      print(calculate_mean_and_variance_manual(total_time))

(0.24803638458251953, 0.24365854894767836)


In [56]:
f(torch.empty([4096,5050,32],device = "cpu"))

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [24]:
total_time = []
t = torch.empty([4096,5050,32],device="cpu")
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
      for i in range(100):
          start_time = time.time()  # 开始计时
          with torch.no_grad():
            soutput = f(t)
          end_time = time.time()  # 结束计时
          
          # 计算并打印函数执行所需的时间
          elapsed_time = end_time - start_time
          total_time.append(elapsed_time * 1000)
      print(calculate_mean_and_variance_manual(total_time))

(2.986135482788086, 190.77499564914433)


### GPU

In [108]:
f(torch.empty([4096,100,32],device = "cuda"))

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')

In [110]:
total_time = []
t = torch.empty([4096,100,32],device="cuda")
for i in range(100):
    start_time = time.time()  # 开始计时
    with torch.no_grad():
      soutput = f(t)
    end_time = time.time()  # 结束计时
    
    # 计算并打印函数执行所需的时间
    elapsed_time = end_time - start_time
    total_time.append(elapsed_time * 1000)
print(calculate_mean_and_variance_manual(total_time))

(0.1461338996887207, 0.016801451755554808)


In [316]:
f(torch.empty([4096,100,64],device = "cuda"))

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')

In [321]:
total_time = []
t = torch.empty([4096,100,64],device="cuda")
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
      for i in range(100):
          start_time = time.time()  # 开始计时
          with torch.no_grad():
            soutput = f(t)
          end_time = time.time()  # 结束计时
          
          # 计算并打印函数执行所需的时间
          elapsed_time = end_time - start_time
          total_time.append(elapsed_time * 1000)
      print(calculate_mean_and_variance_manual(total_time))

(0.16957759857177734, 0.10733138153682376)


In [342]:
f(torch.empty([4096,1000,64],device = "cuda"))

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')

In [349]:
total_time = []
t = torch.empty([4096,1000,64],device="cuda")
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
      for i in range(100):
          start_time = time.time()  # 开始计时
          with torch.no_grad():
            soutput = f(t)
          end_time = time.time()  # 结束计时
          
          # 计算并打印函数执行所需的时间
          elapsed_time = end_time - start_time
          total_time.append(elapsed_time * 1000)
      print(calculate_mean_and_variance_manual(total_time))

(0.17182350158691406, 0.1147689020854159)


In [54]:
f(torch.empty([4096,5050,64],device = "cuda"))

OutOfMemoryError: CUDA out of memory. Tried to allocate 4.93 GiB. GPU 0 has a total capacity of 47.53 GiB of which 175.88 MiB is free. Process 3708359 has 1.50 GiB memory in use. Process 3709323 has 45.28 GiB memory in use. Including non-PyTorch memory, this process has 576.00 MiB memory in use. Of the allocated memory 52.50 MiB is allocated by PyTorch, and 207.50 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [361]:
total_time = []
t = torch.empty([4096,5050,64],device="cuda")
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
      for i in range(100):
          start_time = time.time()  # 开始计时
          with torch.no_grad():
            soutput = f(t)
          end_time = time.time()  # 结束计时
          
          # 计算并打印函数执行所需的时间
          elapsed_time = end_time - start_time
          total_time.append(elapsed_time * 1000)
      print(calculate_mean_and_variance_manual(total_time))

(0.20398855209350586, 0.034293661991569024)


## 修改-不加reorder

In [25]:
@torch.compile()
def f_modify_without_reorder(x):
  empty_like_tensor = torch.ones((4096,100,32))
  result_x = x[0,:50,:] * x[0,:50,:]
  empty_like_tensor[:,:50,:] = result_x
  empty_like_tensor[:,50:,:] = x[0,50:,:] * x[0,50:,:]
#   reorder_x =  empty_like_tensor[:,l_100,:]
  return torch.sum(empty_like_tensor, dim = 1)

### CPU

In [33]:
f_modify_without_reorder(torch.empty([4096,100,32],device = "cpu"))

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [42]:
total_time = []
t = torch.empty([4096,100,32],device="cpu")
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
      for i in range(100):
          start_time = time.time()  # 开始计时
          with torch.no_grad():
            soutput = f_modify_without_reorder(t)
          end_time = time.time()  # 结束计时
          
          # 计算并打印函数执行所需的时间
          elapsed_time = end_time - start_time
          total_time.append(elapsed_time * 1000)
      print(calculate_mean_and_variance_manual(total_time))

(0.2469944953918457, 0.0381317698327166)


In [26]:
@torch.compile()
def f_modify_without_reorder(x):
  empty_like_tensor = torch.ones((4096,5050,32),device="cpu")
  result_x = x[0,:1000,:] * x[0,:1000,:]
  empty_like_tensor[:,:1000,:] = result_x
  empty_like_tensor[:,1000:,:] = x[0,1000:,:] * x[0,1000:,:]
#   reorder_x =  empty_like_tensor[:,l_100,:]
  return torch.sum(empty_like_tensor, dim = 1)

In [27]:
f_modify_without_reorder(torch.empty([4096,5050,32],device = "cpu"))

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [31]:
total_time = []
t = torch.empty([4096,5050,32],device="cpu")
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
      for i in range(100):
          start_time = time.time()  # 开始计时
          with torch.no_grad():
            soutput = f_modify_without_reorder(t)
          end_time = time.time()  # 结束计时
          
          # 计算并打印函数执行所需的时间
          elapsed_time = end_time - start_time
          total_time.append(elapsed_time * 1000)
      print(calculate_mean_and_variance_manual(total_time))

(2.904205322265625, 0.5978259051516943)


In [32]:
@torch.compile()
def f_modify_without_reorder(x):
  empty_like_tensor = torch.ones((4096,5050,32))
  result_x = x[0,:10,:] * x[0,:10,:]
  empty_like_tensor[:,:10,:] = result_x
  empty_like_tensor[:,10:,:] = x[0,10:,:] * x[0,10:,:]
#   reorder_x =  empty_like_tensor[:,l_100,:]
  return torch.sum(empty_like_tensor, dim = 1)

In [34]:
f_modify_without_reorder(torch.empty([4096,5050,32],device = "cpu"))

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [40]:
total_time = []
t = torch.empty([4096,5050,32],device="cpu")
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
      for i in range(100):
          start_time = time.time()  # 开始计时
          with torch.no_grad():
            soutput = f_modify_without_reorder(t)
          end_time = time.time()  # 结束计时
          
          # 计算并打印函数执行所需的时间
          elapsed_time = end_time - start_time
          total_time.append(elapsed_time * 1000)
      print(calculate_mean_and_variance_manual(total_time))

(2.8104615211486816, 0.9702228104004007)


In [70]:
@torch.compile()
def f_modify_without_reorder(x):
  empty_like_tensor = torch.ones((4096,5050,32))
  result_x = x[0,:20,:] * x[0,:20,:]
  empty_like_tensor[:,:20,:] = result_x
  empty_like_tensor[:,20:,:] = x[0,20:,:] * x[0,20:,:]
#   reorder_x =  empty_like_tensor[:,l_100,:]
  return torch.sum(empty_like_tensor, dim = 1)

In [72]:
f_modify_without_reorder(torch.empty([4096,5050,32],device = "cpu"))

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [79]:
total_time = []
t = torch.empty([4096,5050,32],device="cpu")
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
      for i in range(100):
          start_time = time.time()  # 开始计时
          with torch.no_grad():
            soutput = f_modify_without_reorder(t)
          end_time = time.time()  # 结束计时
          
          # 计算并打印函数执行所需的时间
          elapsed_time = end_time - start_time
          total_time.append(elapsed_time * 1000)
      print(calculate_mean_and_variance_manual(total_time))

(2.7547073364257812, 0.6252128408505087)


In [6]:
@torch.compile()
def f_modify_without_reorder(x):
  empty_like_tensor = torch.ones((4096,5050,32))
  result_x = x[0,:50,:] * x[0,:50,:]
  empty_like_tensor[:,:50,:] = result_x
  empty_like_tensor[:,50:,:] = x[0,50:,:] * x[0,50:,:]
#   reorder_x =  empty_like_tensor[:,l_100,:]
  return torch.sum(empty_like_tensor, dim = 1)

In [8]:
f_modify_without_reorder(torch.empty([4096,5050,32],device = "cpu"))

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [10]:
total_time = []
t = torch.empty([4096,5050,32],device="cpu")
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
      for i in range(100):
          start_time = time.time()  # 开始计时
          with torch.no_grad():
            soutput = f_modify_without_reorder(t)
          end_time = time.time()  # 结束计时
          
          # 计算并打印函数执行所需的时间
          elapsed_time = end_time - start_time
          total_time.append(elapsed_time * 1000)
      print(calculate_mean_and_variance_manual(total_time))

(2.8177928924560547, 0.5390379372784082)


## GPU

In [111]:
@torch.compile()
def f_modify_without_reorder(x):
  empty_like_tensor = torch.ones((4096,100,32),device = "cuda")
  result_x = x[0,:50,:] * x[0,:50,:]
  empty_like_tensor[:,:50,:] = result_x
  empty_like_tensor[:,50:,:] = x[0,50:,:] * x[0,50:,:]
#   reorder_x =  empty_like_tensor[:,l_100,:]
  return torch.sum(empty_like_tensor, dim = 1)

In [112]:
f_modify_without_reorder(torch.empty([4096,100,32],device = "cuda"))

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')

In [118]:
total_time = []
t = torch.empty([4096,100,32],device="cuda")
for i in range(100):
    start_time = time.time()  # 开始计时
    soutput = f_modify_without_reorder(t)
    end_time = time.time()  # 结束计时
    
    # 计算并打印函数执行所需的时间
    elapsed_time = end_time - start_time
    total_time.append(elapsed_time * 1000)
print(calculate_mean_and_variance_manual(total_time))

(0.1113271713256836, 0.005528314454750216)


In [323]:
@torch.compile()
def f_modify_without_reorder64(x):
  empty_like_tensor = torch.ones((4096,100,64),device='cuda')
  result_x = x[0,:2,:] * x[0,:2,:]
  empty_like_tensor[:,:2,:] = result_x
  empty_like_tensor[:,2:,:] = x[0,2:,:] * x[0,2:,:]
#   reorder_x =  empty_like_tensor[:,l_100,:]
  return torch.sum(empty_like_tensor, dim = 1)

In [327]:
f_modify_without_reorder64(torch.empty([4096,100,64],device = "cuda"))

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')

In [335]:
total_time = []
t = torch.empty([4096,100,64],device="cuda")
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
      for i in range(100):
          start_time = time.time()  # 开始计时
          with torch.no_grad():
            soutput = f_modify_without_reorder64(t)
          end_time = time.time()  # 结束计时
          
          # 计算并打印函数执行所需的时间
          elapsed_time = end_time - start_time
          total_time.append(elapsed_time * 1000)
      print(calculate_mean_and_variance_manual(total_time))

(0.13831615447998047, 0.11335041679103597)


In [350]:
@torch.compile()
def f_modify_without_reorder64(x):
  empty_like_tensor = torch.ones((4096,1000,64),device='cuda')
  result_x = x[0,:2,:] * x[0,:2,:]
  empty_like_tensor[:,:2,:] = result_x
  empty_like_tensor[:,2:,:] = x[0,2:,:] * x[0,2:,:]
#   reorder_x =  empty_like_tensor[:,l_100,:]
  return torch.sum(empty_like_tensor, dim = 1)

In [354]:
total_time = []
t = torch.empty([4096,1000,64],device="cuda")
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
      for i in range(100):
          start_time = time.time()  # 开始计时
          with torch.no_grad():
            soutput = f_modify_without_reorder64(t)
          end_time = time.time()  # 结束计时
          
          # 计算并打印函数执行所需的时间
          elapsed_time = end_time - start_time
          total_time.append(elapsed_time * 1000)
      print(calculate_mean_and_variance_manual(total_time))

(0.13556241989135742, 0.03458348725757787)


In [363]:
@torch.compile()
def f_modify_without_reorder64(x):
  empty_like_tensor = torch.ones((4096,5050,64),device='cuda')
  result_x = x[0,:2,:] * x[0,:2,:]
  empty_like_tensor[:,:2,:] = result_x
  empty_like_tensor[:,2:,:] = x[0,2:,:] * x[0,2:,:]
#   reorder_x =  empty_like_tensor[:,l_100,:]
  return torch.sum(empty_like_tensor, dim = 1)

In [367]:
total_time = []
t = torch.empty([4096,5050,64],device="cuda")
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
      for i in range(100):
          start_time = time.time()  # 开始计时
          with torch.no_grad():
            soutput = f_modify_without_reorder64(t)
          end_time = time.time()  # 结束计时
          
          # 计算并打印函数执行所需的时间
          elapsed_time = end_time - start_time
          total_time.append(elapsed_time * 1000)
      print(calculate_mean_and_variance_manual(total_time))

(0.14065027236938477, 0.02305010781924466)


In [12]:
@torch.compile()
def f_modify_without_reorder64(x):
  empty_like_tensor = torch.ones((4096,5050,64),device='cuda')
  result_x = x[0,:100,:] * x[0,:100,:]
  empty_like_tensor[:,:100,:] = result_x
  empty_like_tensor[:,100:,:] = x[0,100:,:] * x[0,100:,:]
#   reorder_x =  empty_like_tensor[:,l_100,:]
  return torch.sum(empty_like_tensor, dim = 1)

In [15]:
total_time = []
t = torch.empty([4096,5050,64],device="cuda")
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
      for i in range(100):
          start_time = time.time()  # 开始计时
          with torch.no_grad():
            soutput = f_modify_without_reorder64(t)
          end_time = time.time()  # 结束计时
          
          # 计算并打印函数执行所需的时间
          elapsed_time = end_time - start_time
          total_time.append(elapsed_time * 1000)
      print(calculate_mean_and_variance_manual(total_time))

(0.14458179473876953, 0.1812166433182938)


# 修改加Reorder，但不是乱序

In [89]:
l = [i for i in range(100)]

In [90]:
@torch.compile()
def f_modify_with_reorder_inorder(x):
  empty_like_tensor = torch.ones((4096,100,32))
  result_x = x[0,:2,:] * x[0,:2,:]
  empty_like_tensor[:,:2,:] = result_x
  empty_like_tensor[:,2:,:] = x[0,2:,:] * x[0,2:,:]
#   reorder_x =  empty_like_tensor[:,l_100,:]
  return torch.sum(empty_like_tensor[:,l], dim = 1)

## CPU

In [91]:
f_modify_with_reorder_inorder(torch.empty([4096,100,32],device = "cpu"))

tensor([[ 990000.,  990000.,  990000.,  ..., 1000000., 1000000., 1000000.],
        [ 990000.,  990000.,  990000.,  ..., 1000000., 1000000., 1000000.],
        [ 990000.,  990000.,  990000.,  ..., 1000000., 1000000., 1000000.],
        ...,
        [ 990000.,  990000.,  990000.,  ..., 1000000., 1000000., 1000000.],
        [ 990000.,  990000.,  990000.,  ..., 1000000., 1000000., 1000000.],
        [ 990000.,  990000.,  990000.,  ..., 1000000., 1000000., 1000000.]])

In [92]:
total_time = []
t = torch.empty([4096,100,32],device="cpu")
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
      for i in range(100):
          start_time = time.time()  # 开始计时
          with torch.no_grad():
            soutput = f_modify_with_reorder_inorder(t)
          end_time = time.time()  # 结束计时
          
          # 计算并打印函数执行所需的时间
          elapsed_time = end_time - start_time
          total_time.append(elapsed_time * 1000)
      print(calculate_mean_and_variance_manual(total_time))

(2.7858591079711914, 263.0580623583901)


## GPU

In [93]:
import torch._dynamo
torch._dynamo.config.suppress_errors = True

In [94]:
@torch.compile()
def f_modify_with_reorder_inorder(x):
  empty_like_tensor = torch.ones((4096,100,32),device='cuda')
  result_x = x[0,:2,:] * x[0,:2,:]
  empty_like_tensor[:,:2,:] = result_x
  empty_like_tensor[:,2:,:] = x[0,2:,:] * x[0,2:,:]
#   reorder_x =  empty_like_tensor[:,l_100,:]
  return torch.sum(empty_like_tensor[:,l], dim = 1)

In [96]:
f_modify_with_reorder_inorder(torch.empty([4096,100,32],device = "cuda"))

tensor([[100., 100., 100.,  ..., 100., 100., 100.],
        [100., 100., 100.,  ..., 100., 100., 100.],
        [100., 100., 100.,  ..., 100., 100., 100.],
        ...,
        [100., 100., 100.,  ..., 100., 100., 100.],
        [100., 100., 100.,  ..., 100., 100., 100.],
        [100., 100., 100.,  ..., 100., 100., 100.]], device='cuda:0')

In [99]:
total_time = []
t = torch.empty([4096,100,32],device="cuda")
for i in range(100):
    start_time = time.time()  # 开始计时
    soutput = f_modify_with_reorder_inorder(t)
    end_time = time.time()  # 结束计时
    
    # 计算并打印函数执行所需的时间
    elapsed_time = end_time - start_time
    total_time.append(elapsed_time * 1000)
print(calculate_mean_and_variance_manual(total_time))

(0.11902570724487305, 0.00773259768607204)


In [100]:
l_5050 = [i for i in range(5050)]

In [370]:
@torch.compile()
def f_modify_with_reorder_inorder64_5050(x):
  empty_like_tensor = torch.ones((4096,5050,32),device='cuda')
  result_x = x[0,:2,:] * x[0,:2,:]
  empty_like_tensor[:,:2,:] = result_x
  empty_like_tensor[:,2:,:] = x[0,2:,:] * x[0,2:,:]
#   reorder_x =  empty_like_tensor[:,l_100,:]
  return torch.sum(empty_like_tensor[:,l], dim = 1)

In [374]:
f_modify_with_reorder_inorder64_5050(torch.empty([4096,5050,32],device = "cuda"))

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')

In [379]:
total_time = []
t = torch.empty([4096,5050,32],device="cuda")
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
      for i in range(100):
          start_time = time.time()  # 开始计时
          with torch.no_grad():
            soutput = f_modify_with_reorder_inorder64_5050(t)
          end_time = time.time()  # 结束计时
          
          # 计算并打印函数执行所需的时间
          elapsed_time = end_time - start_time
          total_time.append(elapsed_time * 1000)
      print(calculate_mean_and_variance_manual(total_time))

(0.14266490936279297, 0.03244515457936359)


# 修改加Reorder，是乱序

In [415]:
import random
l_shuffle = [i for i in range(100)]
random.shuffle(l_shuffle)

## Cpu

In [417]:
@torch.compile()
def f_modify_with_reorder_shuffle(x):
  empty_like_tensor = torch.ones((4096,100,32))
  result_x = x[0,:2,:] * x[0,:2,:]
  empty_like_tensor[:,:2,:] = result_x
  empty_like_tensor[:,2:,:] = x[0,2:,:] * x[0,2:,:]
#   reorder_x =  empty_like_tensor[:,l_100,:]
  return torch.sum(empty_like_tensor[:,l_shuffle], dim = 1)

In [419]:
total_time = []
t = torch.empty([4096,100,32],device="cpu")
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
      for i in range(100):
          start_time = time.time()  # 开始计时
          with torch.no_grad():
            soutput = f_modify_with_reorder_shuffle(t)
          end_time = time.time()  # 结束计时
          
          # 计算并打印函数执行所需的时间
          elapsed_time = end_time - start_time
          total_time.append(elapsed_time * 1000)
      print(calculate_mean_and_variance_manual(total_time))

(1.2096595764160156, 0.1504357293015346)


## GPU

In [427]:
@torch.compile()
def f_modify_with_reorder_shuffle64_5050(x):
  empty_like_tensor = torch.ones((4096,5050,64),device='cuda')
  result_x = x[0,:2,:] * x[0,:2,:]
  empty_like_tensor[:,:2,:] = result_x
  empty_like_tensor[:,2:,:] = x[0,2:,:] * x[0,2:,:]
#   reorder_x =  empty_like_tensor[:,l_100,:]
  return torch.sum(empty_like_tensor[:,l_shuffle], dim = 1)

In [ ]:
total_time = []
t = torch.empty([4096,5050,64],device="cuda")
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
      for i in range(100):
          start_time = time.time()  # 开始计时
          with torch.no_grad():
            soutput = f_modify_with_reorder_shuffle64_5050(t)
          end_time = time.time()  # 结束计时
          
          # 计算并打印函数执行所需的时间
          elapsed_time = end_time - start_time
          total_time.append(elapsed_time * 1000)
      print(calculate_mean_and_variance_manual(total_time))

(0.1744842529296875, 0.4745138627640699)


: 

# sum

In [173]:
@torch.compile
def f(x):
#   reorder_x =  empty_like_tensor[:,l_100,:]
  return torch.sum(x * x, dim = 1)

In [176]:
f(torch.empty([4096,100,64],device = "cpu"))
total_time = []
t = torch.ones([4096,100,64],device="cpu")
for i in range(100):
    start_time = time.time()  # 开始计时
    with torch.no_grad():
      soutput = f(t)
    end_time = time.time()  # 结束计时
    
    # 计算并打印函数执行所需的时间
    elapsed_time = end_time - start_time
    total_time.append(elapsed_time * 1000)
print(calculate_mean_and_variance_manual(total_time))

(0.9696149826049805, 0.05414658860445343)


In [177]:
@torch.compile()
def f_modify_without_reorder(x):
  x = x * x
#   reorder_x =  empty_like_tensor[:,l_100,:]
  return torch.sum(x[0,:50],dim = 0)+torch.sum(x[:,50:],dim = 1)

In [180]:
f_modify_without_reorder(torch.empty([4096,100,32],device = "cpu"))
total_time = []
t = torch.ones([4096,100,64],device="cpu")
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
      for i in range(100):
          start_time = time.time()  # 开始计时
          with torch.no_grad():
            soutput = f_modify_without_reorder(t)
          end_time = time.time()  # 结束计时
          
          # 计算并打印函数执行所需的时间
          elapsed_time = end_time - start_time
          total_time.append(elapsed_time * 1000)
      print(calculate_mean_and_variance_manual(total_time))

(0.3399801254272461, 0.12991782793960738)


In [181]:
@torch.compile()
def f_modify_without_reorder(x):
  x_r = x[0,:50] * x[0,:50]
  x_n = x[:,50:] * x[:,50:]
#   reorder_x =  empty_like_tensor[:,l_100,:]
  return torch.sum(x_r,dim = 0)+torch.sum(x_n,dim = 1)

In [186]:
f_modify_without_reorder(torch.empty([4096,100,32],device = "cpu"))
total_time = []
t = torch.ones([4096,100,64],device="cpu")
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
      for i in range(100):
          start_time = time.time()  # 开始计时
          with torch.no_grad():
            soutput = f_modify_without_reorder(t)
          end_time = time.time()  # 结束计时
          
          # 计算并打印函数执行所需的时间
          elapsed_time = end_time - start_time
          total_time.append(elapsed_time * 1000)
      print(calculate_mean_and_variance_manual(total_time))

(0.6042313575744629, 0.20809929998790722)


In [224]:
@torch.compile
def f(x):
#   reorder_x =  empty_like_tensor[:,l_100,:]
  return torch.sum(x * x, dim = 1)

In [229]:
f(torch.empty([4096,1000,64],device = "cuda"))
total_time = []
t = torch.ones([4096,1000,64],device="cuda")
for i in range(100):
    start_time = time.time()  # 开始计时
    with torch.no_grad():
      soutput = f(t)
    end_time = time.time()  # 结束计时
    
    # 计算并打印函数执行所需的时间
    elapsed_time = end_time - start_time
    total_time.append(elapsed_time * 1000)
print(calculate_mean_and_variance_manual(total_time))

(0.13529062271118164, 0.003427034442893273)


In [230]:
@torch.compile()
def f_modify_without_reorder(x):
  x = x * x
#   reorder_x =  empty_like_tensor[:,l_100,:]
  return torch.sum(x[0,:500],dim = 0)+torch.sum(x[:,500:],dim = 1)

In [233]:
f_modify_without_reorder(torch.empty([4096,1000,64],device = "cuda"))
total_time = []
t = torch.ones([4096,1000,64],device="cuda")
for i in range(100):
    start_time = time.time()  # 开始计时
    with torch.no_grad():
      soutput = f_modify_without_reorder(t)
    end_time = time.time()  # 结束计时
    
    # 计算并打印函数执行所需的时间
    elapsed_time = end_time - start_time
    total_time.append(elapsed_time * 1000)
print(calculate_mean_and_variance_manual(total_time))

(0.12399911880493164, 0.0004035849713091011)


In [234]:
@torch.compile()
def f_modify_without_reorder(x):
  x_r = x[0,:500] * x[0,:500]
  x_n = x[:,500:] * x[:,500:]
#   reorder_x =  empty_like_tensor[:,l_100,:]
  return torch.sum(x_r,dim = 0)+torch.sum(x_n,dim = 1)

In [237]:
f_modify_without_reorder(torch.empty([4096,1000,64],device = "cuda"))
total_time = []
t = torch.ones([4096,1000,64],device="cuda")
for i in range(100):
    start_time = time.time()  # 开始计时
    with torch.no_grad():
      soutput = f_modify_without_reorder(t)
    end_time = time.time()  # 结束计时
    
    # 计算并打印函数执行所需的时间
    elapsed_time = end_time - start_time
    total_time.append(elapsed_time * 1000)
print(calculate_mean_and_variance_manual(total_time))

(0.12708425521850586, 0.002094229165550132)


In [238]:
@torch.compile()
def f_modify_without_reorder(x):
    empty_x = torch.empty([4096,1000,64],device="cuda")
    empty_x[:,:500] = x[0,:500] * x[0,:500]
    empty_x[:,500:] = x[:,500:] * x[:,500:]
#   reorder_x =  empty_like_tensor[:,l_100,:]
    return torch.sum(empty_x,dim = 1)

In [243]:
f_modify_without_reorder(torch.empty([4096,1000,64],device = "cuda"))
total_time = []
t = torch.ones([4096,1000,64],device="cuda")
for i in range(100):
    start_time = time.time()  # 开始计时
    with torch.no_grad():
      soutput = f_modify_without_reorder(t)
    end_time = time.time()  # 结束计时
    
    # 计算并打印函数执行所需的时间
    elapsed_time = end_time - start_time
    total_time.append(elapsed_time * 1000)
print(calculate_mean_and_variance_manual(total_time))

(0.09167671203613281, 0.0018779593119688798)


In [244]:
@torch.compile()
def f_modify_without_reorder(x):
    empty_x = torch.empty([4096,1000,64],device="cuda")
    empty_x[:,:200] = x[0,:200] * x[0,:200]
    empty_x[:,200:] = x[:,200:] * x[:,200:]
#   reorder_x =  empty_like_tensor[:,l_100,:]
    return torch.sum(empty_x,dim = 1)

In [250]:
f_modify_without_reorder(torch.empty([4096,1000,64],device = "cuda"))
total_time = []
t = torch.ones([4096,1000,64],device="cuda")
for i in range(100):
    start_time = time.time()  # 开始计时
    with torch.no_grad():
      soutput = f_modify_without_reorder(t)
    end_time = time.time()  # 结束计时
    
    # 计算并打印函数执行所需的时间
    elapsed_time = end_time - start_time
    total_time.append(elapsed_time * 1000)
print(calculate_mean_and_variance_manual(total_time))

(0.0942540168762207, 0.001045875768568294)


目前：
1.Element wise算子使用新的改写方法 GPU上有效
2.reduce类在CPU GPU上有

In [206]:
@torch.compile
def f(x):
#   reorder_x =  empty_like_tensor[:,l_100,:]
  return torch.sum(x * x, dim = 1)

In [210]:
f(torch.empty([4096,100,64],device = "cuda"))
total_time = []
t = torch.ones([4096,100,64],device="cuda")
for i in range(100):
    start_time = time.time()  # 开始计时
    with torch.no_grad():
      soutput = f(t)
    end_time = time.time()  # 结束计时
    
    # 计算并打印函数执行所需的时间
    elapsed_time = end_time - start_time
    total_time.append(elapsed_time * 1000)
print(calculate_mean_and_variance_manual(total_time))

(0.1303386688232422, 0.003033825214515673)


In [211]:
@torch.compile()
def f_modify_without_reorder(x):
  x_r = x[0,:50] * x[0,:50]
  x_n = x[:,50:] * x[:,50:]
#   reorder_x =  empty_like_tensor[:,l_100,:]
  return torch.sum(x_r,dim = 0)+torch.sum(x_n,dim = 1)

In [214]:
f_modify_without_reorder(torch.empty([4096,100,64],device = "cuda"))
total_time = []
t = torch.ones([4096,1000,64],device="cuda")
for i in range(100):
    start_time = time.time()  # 开始计时
    with torch.no_grad():
      soutput = f_modify_without_reorder(t)
    end_time = time.time()  # 结束计时
    
    # 计算并打印函数执行所需的时间
    elapsed_time = end_time - start_time
    total_time.append(elapsed_time * 1000)
print(calculate_mean_and_variance_manual(total_time))

(0.15122652053833008, 0.0020803682843961724)


In [216]:
@torch.compile()
def f_modify_without_reorder(x):
    empty_x = torch.empty([4096,100,64],device="cuda")
    empty_x[:,:50] = x[0,:50] * x[0,:50]
    empty_x[:,50:] = x[:,50:] * x[:,50:]
#   reorder_x =  empty_like_tensor[:,l_100,:]
    return torch.sum(empty_x,dim = 1)

In [223]:
f_modify_without_reorder(torch.empty([4096,100,64],device = "cuda"))
total_time = []
t = torch.ones([4096,100,64],device="cuda")
for i in range(100):
    start_time = time.time()  # 开始计时
    with torch.no_grad():
      soutput = f_modify_without_reorder(t)
    end_time = time.time()  # 结束计时
    
    # 计算并打印函数执行所需的时间
    elapsed_time = end_time - start_time
    total_time.append(elapsed_time * 1000)
print(calculate_mean_and_variance_manual(total_time))

(0.12250185012817383, 0.0013924496840900247)


In [16]:
@torch.compile
def f(x):
#   reorder_x =  empty_like_tensor[:,l_100,:]
  return torch.sum(x ** 2, dim = 1)

In [19]:
f(torch.empty([4096,1009,64],device = "cuda"))
total_time = []
t = torch.ones([4096,1000,64],device="cuda")
for i in range(100):
    start_time = time.time()  # 开始计时
    with torch.no_grad():
      soutput = f(t)
    end_time = time.time()  # 结束计时
    
    # 计算并打印函数执行所需的时间
    elapsed_time = end_time - start_time
    total_time.append(elapsed_time * 1000)
print(calculate_mean_and_variance_manual(total_time))

(0.10809183120727539, 0.0026601431443395995)


In [22]:
@torch.compile()
def f_modify_without_reorder(x):
    empty_x = torch.empty([4096,1000,64],device="cuda")
    empty_x[:,:500] = x[0,:500] **2
    empty_x[:,500:] = x[:,500:] **2
#   reorder_x =  empty_like_tensor[:,l_100,:]
    return torch.sum(empty_x,dim = 1)

In [32]:
f_modify_without_reorder(torch.empty([4096,1000,64],device = "cuda"))
total_time = []
t = torch.ones([4096,1000,64],device="cuda")
for i in range(100):
    start_time = time.time()  # 开始计时
    with torch.no_grad():
      soutput = f_modify_without_reorder(t)
    end_time = time.time()  # 结束计时
    
    # 计算并打印函数执行所需的时间
    elapsed_time = end_time - start_time
    total_time.append(elapsed_time * 1000)
print(calculate_mean_and_variance_manual(total_time))

(0.09702444076538086, 0.003024003234486372)
